# BERT's playground 
Hello there! Welcome on BERT's playground. You may play with BERT here and see what he can do but always make sure he feels respected and admired.

## Setups


In [3]:
import sys
sys.path.append('../ml/BERT')
import masking
import BERT
from Vectorisation import Vectorisation
from Config import Config
from MaskedTextGenerator import MaskedTextGenerator

import pickle
import tensorflow as tf
import numpy as np
from tensorflow import keras

with open("../../data/ml4science_data.pkl", "rb") as fp:
    data_dict = pickle.load(fp)

config = Config()
vec = Vectorisation(config=config)

# Testing BERT

In [4]:
# Prepare data for masked language model
encoded = vec.encode_dict(data_dict)
x_masked_encoded, y_masked_encoded, sample_weights = masking.mask_input_and_labels(encoded, config.TOKEN_DICT, seed=32)
print(x_masked_encoded.shape, y_masked_encoded.shape, sample_weights.shape)

mlm_ds = tf.data.Dataset.from_tensor_slices((x_masked_encoded, y_masked_encoded, sample_weights))
mlm_ds = mlm_ds.shuffle(1000).batch(config.BATCH_SIZE)

print(mlm_ds)

(254, 128) (254, 128) (254, 128)
<_BatchDataset element_spec=(TensorSpec(shape=(None, 128), dtype=tf.int32, name=None), TensorSpec(shape=(None, 128), dtype=tf.int32, name=None), TensorSpec(shape=(None, 128), dtype=tf.float64, name=None))>


In [5]:
encoded

array([[26,  2, 26, ...,  0,  0,  0],
       [26, 21, 26, ..., 14, 11, 14],
       [25, 26, 21, ...,  0,  0,  0],
       ...,
       [26,  2, 26, ...,  0,  0,  0],
       [26, 25, 26, ...,  0,  0,  0],
       [26,  2, 26, ..., 22, 26, 21]])

In [6]:
sample_tokens = x_masked_encoded[0:1]
print(y_masked_encoded[0])

[26  2 26 21 21 26 23 25 21 23  2 21 26 21 21 26 21 25  3  8  3  3  3  3
  3  3  3  3  7  8  4  2  8  3 16 20 21 26 21 23 21  3  8  7  8  3  5  8
  4  8  2  8  3  8 10  9 10  9  9 15 20 11  9 14 13 11  9 14 13 14 10 11
  9  9 13 14 13 14 11  9 10 14 13 10 11  9 14  9 14 15 20 15 15 17 20  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0]


In [7]:
print(x_masked_encoded[0])

[26  2 26 21 21 26  9 25 21 23  2 21 26 21 21 26 21  1  3  8  3  1  3  3
  3  3  3  3  7  8  1  2  8  3 16 20 21 26 21 23 21  3  8  7  8  3  5  8
  4  1  2  8  3  1 10  9 10  9  9 15  1 11  9  1  1 11  9 14 13 14 10  1
  9  9 13 14 13 14 11  9 10 14 13 10 11  1 14  9  1 15 20 15 15 17 20  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0]


In [8]:
generator_callback = MaskedTextGenerator(sample_tokens, config.TOKEN_DICT['[MASK]'])

bert_masked_model = BERT.create_masked_language_bert_model(config)
bert_masked_model.summary()

Model: "masked_bert_model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128)]                0         []                            
                                                                                                  
 word_embedding (Embedding)  (None, 128, 64)              1728      ['input_1[0][0]']             
                                                                                                  
 tf.__operators__.add (TFOp  (None, 128, 64)              0         ['word_embedding[0][0]']      
 Lambda)                                                                                          
                                                                                                  
 encoder_0/multiheadattenti  (None, 128, 64)              16640     ['tf.__operato

In [9]:
# A callback in Keras is a function that is called at certain points during the training process. -> Here is called after each epoch during the training
# Here we use to see the "performance" at each epoch while predicting on a "test set" aka the sample_tokens

bert_masked_model.fit(mlm_ds, epochs=20)#, callbacks=[generator_callback]) 

Epoch 1/20


16/16 [==============================] - 2s 18ms/step - loss: 3.2364
Epoch 2/20
16/16 [==============================] - 0s 16ms/step - loss: 2.7211
Epoch 3/20
16/16 [==============================] - 0s 15ms/step - loss: 2.6188
Epoch 4/20
16/16 [==============================] - 0s 13ms/step - loss: 2.5628
Epoch 5/20
16/16 [==============================] - 0s 17ms/step - loss: 2.5261
Epoch 6/20
16/16 [==============================] - 0s 15ms/step - loss: 2.5169
Epoch 7/20
16/16 [==============================] - 0s 15ms/step - loss: 2.4786
Epoch 8/20
16/16 [==============================] - 0s 15ms/step - loss: 2.4564
Epoch 9/20
16/16 [==============================] - 0s 15ms/step - loss: 2.4502
Epoch 10/20
16/16 [==============================] - 0s 16ms/step - loss: 2.3838
Epoch 11/20
16/16 [==============================] - 0s 16ms/step - loss: 2.3181
Epoch 12/20
16/16 [==============================] - 0s 17ms/step - loss: 2.2010
Epoch 13/20
16/16 [=============================

In [10]:
predictions = bert_masked_model.predict(x_masked_encoded[0:1])

predictions_max = np.argmax(predictions, axis=2)
print("Predictions:\n",predictions_max)
print("Original:\n", y_masked_encoded[0:1])
print("Masked:\n", x_masked_encoded[0:1])

1/1 [==============================] - 0s 175ms/step
Predictions:
 [[26 26 26 21 21 26 21 21 21 21 21 21 26 21 21 26 21 26 26 21 21  9  9  9
   9  9  9  9  9  9  9 14  9  9 16 20 21 26 21  9 21  9 11  9  8 15 14  8
   9  9 14  8  9  9 10  9 10  9 14 14 14 14 10 14 14 14  9 14 10 14 10  9
   9  9  9 14 10 14 14  9 10 14  9  9  9  9 14  9 14 15 20 16 16 20 20 20
  20 20 14 14 14 14 14 20 20 20 20 20 20 20  4  4 15 15 15 15 15  4  4  4
   4  9 14  4  4 14  9  9]]
Original:
 [[26  2 26 21 21 26 23 25 21 23  2 21 26 21 21 26 21 25  3  8  3  3  3  3
   3  3  3  3  7  8  4  2  8  3 16 20 21 26 21 23 21  3  8  7  8  3  5  8
   4  8  2  8  3  8 10  9 10  9  9 15 20 11  9 14 13 11  9 14 13 14 10 11
   9  9 13 14 13 14 11  9 10 14 13 10 11  9 14  9 14 15 20 15 15 17 20  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0]]
Masked:
 [[26  2 26 21 21 26  9 25 21 23  2 21 26 21 21 26 21  1  3  8  3  1  3  3
   3  3  3  3  7  8  1  2  8  3 16 20 21 26

In [11]:
print(predictions_max == y_masked_encoded[0:1])
# print accuracy
print("Accuracy brut: ", np.sum(predictions_max == y_masked_encoded[0:1]) / (100*len(y_masked_encoded[0:1])))
# print accuracy without padding
print("Accuracy without taking in acount padding: ", np.sum((predictions_max == y_masked_encoded[0:1]) * (y_masked_encoded[0:1] != 0)) / np.sum(y_masked_encoded[0:1] != 0))
print("Accuracy on masked tokens: ", np.sum((predictions_max == y_masked_encoded[0:1]) * (x_masked_encoded[0:1] == config.TOKEN_DICT['[MASK]'])) / np.sum(x_masked_encoded[0:1] == config.TOKEN_DICT['[MASK]']))

[[ True False  True  True  True  True False False  True False False  True
   True  True  True  True  True False False False False False False False
  False False False False False False False False False False  True  True
   True  True  True False  True False False False  True False False  True
  False False False  True False False  True  True  True  True False False
  False False False  True False False  True  True False  True  True False
   True  True False  True False  True False  True  True  True False False
  False  True  True  True  True  True  True False False False  True False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False]]
Accuracy brut:  0.44
Accuracy without taking in acount padding:  0.4631578947368421
Accuracy on masked tokens:  0.2727272727272727


In [12]:
# This how we can load a Keras model
"""# Load OUR pretrained bert model
mlm_model = keras.models.load_model(
    "bert_mlm_imdb.keras", custom_objects={"MaskedLanguageModel": MaskedLanguageModel}
)"""

# Kinda failed attempt to create a end to end model (we don't really need it) -> But it's a good example of how to create a model with a custom loss function and reshape the output
"""def get_end_to_end(model):
    inputs = keras.Input(shape=(None,))
    outputs = model(inputs)
    reshaped_outputs = keras.layers.Lambda(lambda x: keras.backend.argmax(x, axis=-1))(outputs)
    end_to_end_model = keras.Model(inputs, reshaped_outputs, name="end_to_end_model")
    optimizer = keras.optimizers.Adam(learning_rate=config.bert.LR)
    end_to_end_model.compile(
        optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"]
    )
    return end_to_end_model

end_to_end_classification_model = get_end_to_end(bert_masked_model)

# Build dataset for end to end model input (will be used at the end)
test_raw_classifier_ds = tf.data.Dataset.from_tensor_slices((x_masked_encoded, y_masked_encoded)).batch(config.BATCH_SIZE)

end_to_end_classification_model.evaluate(test_raw_classifier_ds)"""

'def get_end_to_end(model):\n    inputs = keras.Input(shape=(None,))\n    outputs = model(inputs)\n    reshaped_outputs = keras.layers.Lambda(lambda x: keras.backend.argmax(x, axis=-1))(outputs)\n    end_to_end_model = keras.Model(inputs, reshaped_outputs, name="end_to_end_model")\n    optimizer = keras.optimizers.Adam(learning_rate=config.bert.LR)\n    end_to_end_model.compile(\n        optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"]\n    )\n    return end_to_end_model\n\nend_to_end_classification_model = get_end_to_end(bert_masked_model)\n\n# Build dataset for end to end model input (will be used at the end)\ntest_raw_classifier_ds = tf.data.Dataset.from_tensor_slices((x_masked_encoded, y_masked_encoded)).batch(config.BATCH_SIZE)\n\nend_to_end_classification_model.evaluate(test_raw_classifier_ds)'

# Testing BertPipeline

In [13]:
# Load the sequences used in src\script_oversample.py
with open('../../data/sequences.pkl', 'rb') as f:
    sequences = pickle.load(f)

sequences[0][0:3]

[[0, 0, 0, 1, 0, 0, 0, 0, 0, 7.889],
 [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 75.141, 0.0],
 [0.0, 0.0, 0.0, 1.0, 0, 0, 0, 0, 0, 3.25]]

In [14]:
# Test with not SEP because unsing only sequences not info of dictionarry
sequences = vec.encode(sequences)
print(sequences.shape,sequences[0])

(254, 128) [26 25 26 21 21 26 23 25 21 23 25 21 26 21 21 26 21 25  3  8  3  3  3  3
  3  3  3  3  7  8  4  7  8  3 16 20 21 26 21 23 21  3  8  7  8  3  5  8
  4  8  7  8  3  8 10  9 10  9  9 15 20 11  9 14 13 11  9 14 13 14 10 11
  9  9 13 14 13 14 11  9 10 14 13 10 11  9 14  9 14 15 20 15 15 17 20  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0]


In [15]:
x_masked_encoded, y_masked_encoded, sample_weights = masking.mask_input_and_labels(sequences, config.TOKEN_DICT)
print(x_masked_encoded.shape, y_masked_encoded.shape, sample_weights.shape)

mlm_ds = tf.data.Dataset.from_tensor_slices((x_masked_encoded, y_masked_encoded, sample_weights))
mlm_ds = mlm_ds.shuffle(1000).batch(config.BATCH_SIZE)
print(mlm_ds)



(254, 128) (254, 128) (254, 128)
<_BatchDataset element_spec=(TensorSpec(shape=(None, 128), dtype=tf.int32, name=None), TensorSpec(shape=(None, 128), dtype=tf.int32, name=None), TensorSpec(shape=(None, 128), dtype=tf.float64, name=None))>


In [16]:
bert_masked_model = BERT.create_masked_language_bert_model(config)
#bert_masked_model.summary()

bert_masked_model.fit(mlm_ds, config.bert.epoch)

ValueError: `y` argument is not supported when using dataset as input.

In [ ]:
from BertPipeline import BertPipeline

bert = BertPipeline(config, vec)
bert_masked_model = bert.train(sequences)

new_sequences = bert.predict(encoded, bert_masked_model)

Model: "masked_bert_model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 128)]                0         []                            
                                                                                                  
 word_embedding (Embedding)  (None, 128, 64)              1728      ['input_4[0][0]']             
                                                                                                  
 tf.__operators__.add_9 (TF  (None, 128, 64)              0         ['word_embedding[0][0]']      
 OpLambda)                                                                                        
                                                                                                  
 encoder_0/multiheadattenti  (None, 128, 64)              16640     ['tf.__operato

ValueError: `y` argument is not supported when using dataset as input.